In [15]:
import pandas as pd
import ijson
import json
import codecs
import os

Trying to extract sample out of 28 GB JSON file by parsing with ijson.

JSON structure:

{"articles": [
	{"abstractText":"text..", "journal":"journal..", "meshMajor":["mesh1",...,"meshN"], "pmid":"PMID", "title":"title..", "year":"YYYY"},
	..., 
	{..}
]}

Wegen ***IncompleteJSONError*** muss File umcodiert werden. Die Funktion liest die Datei in Chunks und versucht, jeden Chunk als UTF-8 zu dekodieren. Ungültige Byte-Sequenzen werden durch das Unicode-Ersatzzeichen ersetzt.

In [11]:
filename = "data/allMeSH_2022/allMeSH_2022.json"

def clean_json_file(input_filename, output_filename):
    total_size = os.path.getsize(input_filename)
    processed_size = 0

    with open(input_filename, 'rb') as input_file, open(output_filename, 'wb') as output_file:
        while True:
            chunk = input_file.read(1024 * 1024)  # Lese die Datei in 1MB Chunks
            if not chunk:
                break  # Beende, wenn das Ende der Datei erreicht ist

            clean_chunk = chunk.decode('utf-8', errors='replace').encode('utf-8')
            output_file.write(clean_chunk)

            processed_size += len(chunk)
            progress = (processed_size / total_size) * 100
            print(f"\rFortschritt: {progress:.2f}%", end='')

    print("\nBereinigung abgeschlossen.")

clean_json_file(filename, 'bereinigte_datei.json')


Fortschritt: 100.00%
Bereinigung abgeschlossen.


Extrahieren eines Sample-Chunks aus der 28 GB JSON Datei

In [19]:
filename = "data/bereinigte_datei.json"
articles = []
max_articles = 1000

with open(filename, 'rb') as input_file:
    # Hier passen wir den Pfad an, um die Artikel direkt zu erreichen
    parser = ijson.items(input_file, 'articles.item')
    for article in parser:
        articles.append(article)
        if len(articles) >= max_articles:
            break

# speichern des samples

# Speichere die extrahierten Artikel in einer neuen JSON-Datei
with open('data/10k_sample_article.json', 'w', encoding='utf-8') as output_file:
    json.dump(articles, output_file, ensure_ascii=False, indent=4)
    
print(f"{len(articles)} Artikel wurden in '10k_sample_article.json' gespeichert.")

1000 Artikel wurden in '10k_sample_article.json' gespeichert.


#### MeSH

Durch MeSH sollen Beziehungen bzw. Kanten im Graphen Modelliert werden.

- MeSH (Medical Subject Headings) is the NLM controlled vocabulary thesaurus used for indexing articles for PubMed.

Überprüfung ob alle Artikel ein Attribut "meshMajor" mit mindestens 1 Eintrag haben.

In [20]:
# Datei laden, die die extrahierten Artikel enthält
filename = 'data/10k_sample_article.json'

with open(filename, 'r', encoding='utf-8') as file:
    articles = json.load(file)

# Überprüfe jedes Artikelobjekt auf das Vorhandensein und den Inhalt des "meshMajor"-Attributs
missing_or_empty_meshMajor = [article for article in articles if not article.get('meshMajor')]

# Ausgabe der Ergebnisse
if missing_or_empty_meshMajor:
    print(f"{len(missing_or_empty_meshMajor)} von {len(articles)} Artikeln haben kein 'meshMajor'-Attribut oder es ist leer.")
else:
    print("Alle Artikel haben das 'meshMajor'-Attribut mit mindestens einem Eintrag.")

Alle Artikel haben das 'meshMajor'-Attribut mit mindestens einem Eintrag.


#### First Import in Neo4j

In [21]:
from neo4j import GraphDatabase

ModuleNotFoundError: No module named 'neo4j'